In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset/dakshina_dataset_v1.0/README.md
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.test.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.train.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/lexicons/mr.translit.sampled.dev.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.train.text.sorted.tsv/wiki-filt.train.text.sorted.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-full.nonblock.sections.list.txt/wiki-full.nonblock.sections.list.txt
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-filt.valid.text.shuf.txt/wiki-filt.valid.text.shuf.txt
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-full.urls.tsv/wiki-full.urls.tsv
/kaggle/input/dataset/dakshina_dataset_v1.0/mr/native_script_wikipedia/mr.wiki-full.omit_pages.txt/wiki-full.omit_pages.txt
/kaggle/input/dataset/dakshina_data

In [2]:
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import wandb
import matplotlib.pyplot as plt
from tqdm import tqdm
from types import SimpleNamespace
import logging
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import time
import re

In [3]:
wandb.login(key="367b223a3f0954311018074733b3ed8dc409c2ff")
# Constants
wandb_project = "DL_A3_seq2seq"
wandb_entity= "da24m006-iit-madras"  

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da24m006 (da24m006-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
data_dir = '/kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons'
traindata = pd.read_csv(f'{data_dir}/hi.translit.sampled.train.tsv',sep='\t',names=['native', 'latin', 'count'],usecols=[0, 1])
valdata= pd.read_csv(f'{data_dir}/hi.translit.sampled.dev.tsv',sep='\t',names=['native', 'latin', 'count'],usecols=[0, 1])
testdata = pd.read_csv(f'{data_dir}/hi.translit.sampled.test.tsv',sep='\t',names=['native', 'latin', 'count'],usecols=[0, 1])

In [6]:
traindata

,native,latin
0,अं,an
1,अंकगणित,ankganit
2,अंकल,uncle
3,अंकुर,ankur
4,अंकुरण,ankuran
...,...,...
44199,ह्वेनसांग,hiuentsang
44200,ह्वेनसांग,hsuantsang
44201,ह्वेनसांग,hyensang
44202,ह्वेनसांग,xuanzang


In [7]:
print("Total Dataset: ")
print("Total train data:", len(traindata))
print("Total validation data:", len(valdata))
print("Totat test data:", len(testdata))

Total Dataset: 
Total train data: 44204
Total validation data: 4358
Totat test data: 4502


In [8]:
Hindi_vocab = set()
max_hin_len = 0

# Process training data
for x in range(len(traindata)):
    native_word = traindata.iloc[x]['native']
    if pd.isna(native_word):
        continue

    max_hin_len = max(max_hin_len, len(native_word))
    for char in native_word:
        Hindi_vocab.add(char)

# Process test data
for x in range(len(testdata)):
    native_word = testdata.iloc[x]['native']
    if pd.isna(native_word):
        continue

    for char in native_word:
        if char not in Hindi_vocab:
            print(char)
            Hindi_vocab.add(char)


Hindi_vocab = sorted(Hindi_vocab)

print("Max Hindi word length:", max_hin_len)
print("Total unique Hindi characters:", len(Hindi_vocab))
print(Hindi_vocab)

Max Hindi word length: 19
Total unique Hindi characters: 63
['ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॅ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ']


In [9]:
max_eng_len = 0
English_vocab = set() 

for x in range(len(traindata)):
    latin_word = traindata.iloc[x]['latin']
    
    if pd.isna(latin_word):
        continue  # skip if value is missing

    word_len = len(latin_word)
    max_eng_len = max(max_eng_len, word_len)

    for char in latin_word:
        English_vocab.add(char)

# Convert set to sorted list
English_vocab = sorted(English_vocab)
print("Max english word length:", max_eng_len)
print("Total unique english characters:", len(English_vocab))
print(English_vocab)


Max english word length: 20
Total unique english characters: 26
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Dataset Preprocessing

In [10]:
class TransliterationDataset(Dataset):
    def __init__(self, file_path):
        self.source_texts = []
        self.target_texts = []
        
        # Check if file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Data file not found: {file_path}")
            
        with open(file_path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                parts = line.strip().split('\t')
                
                # Handle different file formats
                if len(parts) == 3:  # Format: Hindi\tRomanized\tFrequency
                    target, source, _ = parts
                    self.source_texts.append(source)
                    self.target_texts.append(target)
                elif len(parts) == 2:  # Format: Hindi\tRomanized
                    target, source = parts
                    self.source_texts.append(source)
                    self.target_texts.append(target)
                else:
                    print(f"Warning: Line {line_num} has unexpected format (expected 2 or 3 columns): {line.strip()}")
        
        # If no data was loaded, raise an error
        if len(self.source_texts) == 0:
            raise ValueError(f"No data was loaded from {file_path}. File may be empty or incorrectly formatted.")
                    
        self.source_char_set = set()
        self.target_char_set = set()
        
        for source in self.source_texts:
            self.source_char_set.update(source)
        
        for target in self.target_texts:
            self.target_char_set.update(target)
        
        self.source_char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(self.source_char_set))}
        self.source_idx_to_char = {idx + 1: char for idx, char in enumerate(sorted(self.source_char_set))}
        self.source_char_to_idx['<PAD>'] = 0
        self.source_idx_to_char[0] = '<PAD>'
        self.source_char_to_idx['< SOS >'] = len(self.source_char_to_idx)
        self.source_idx_to_char[len(self.source_idx_to_char)] = '< SOS >'
        self.source_char_to_idx['<EOS>'] = len(self.source_char_to_idx)
        self.source_idx_to_char[len(self.source_idx_to_char)] = '<EOS>'
        
        self.target_char_to_idx = {char: idx + 1 for idx, char in enumerate(sorted(self.target_char_set))}
        self.target_idx_to_char = {idx + 1: char for idx, char in enumerate(sorted(self.target_char_set))}
        self.target_char_to_idx['<PAD>'] = 0
        self.target_idx_to_char[0] = '<PAD>'
        self.target_char_to_idx['< SOS >'] = len(self.target_char_to_idx)
        self.target_idx_to_char[len(self.target_idx_to_char)] = '< SOS >'
        self.target_char_to_idx['<EOS>'] = len(self.target_char_to_idx)
        self.target_idx_to_char[len(self.target_idx_to_char)] = '<EOS>'
        
    def __len__(self):
        return len(self.source_texts)
    
    def __getitem__(self, idx):
        source = self.source_texts[idx]
        target = self.target_texts[idx]
        
        source_indices = [self.source_char_to_idx['< SOS >']] + [self.source_char_to_idx[char] for char in source] + [self.source_char_to_idx['<EOS>']]
        target_indices = [self.target_char_to_idx['< SOS >']] + [self.target_char_to_idx[char] for char in target] + [self.target_char_to_idx['<EOS>']]
        
        return {
            'source': source,
            'target': target,
            'source_indices': source_indices,
            'target_indices': target_indices,
            'source_length': len(source_indices),
            'target_length': len(target_indices)
        }
    
    @property
    def source_vocab_size(self):
        return len(self.source_char_to_idx)
    
    @property
    def target_vocab_size(self):
        return len(self.target_char_to_idx)

def collate_fn(batch):
    max_source_len = max([len(item['source_indices']) for item in batch])
    max_target_len = max([len(item['target_indices']) for item in batch])
    
    source_batch = []
    target_batch = []
    source_lengths = []
    target_lengths = []
    
    source_texts = []
    target_texts = []
    
    for item in batch:
        source_padded = item['source_indices'] + [0] * (max_source_len - len(item['source_indices']))
        target_padded = item['target_indices'] + [0] * (max_target_len - len(item['target_indices']))
        
        source_batch.append(source_padded)
        target_batch.append(target_padded)
        source_lengths.append(item['source_length'])
        target_lengths.append(item['target_length'])
        
        source_texts.append(item['source'])
        target_texts.append(item['target'])
    
    return {
        'source': torch.LongTensor(source_batch),
        'target': torch.LongTensor(target_batch),
        'source_lengths': torch.LongTensor(source_lengths),
        'target_lengths': torch.LongTensor(target_lengths),
        'source_texts': source_texts,
        'target_texts': target_texts
    }

def get_dataloader(file_path, batch_size, shuffle=True):
    try:
        dataset = TransliterationDataset(file_path)
        dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=shuffle,
            collate_fn=collate_fn
        )
        return dataloader, dataset
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        print(f"Please make sure the Dakshina dataset is downloaded and located correctly.")
        print(f"Expected file path: {file_path}")
        print(f"Current working directory: {os.getcwd()}")
        raise
    except ValueError as e:
        print(f"Error: {str(e)}")
        raise

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class Encoder(nn.Module):
    def __init__(self, input_size, embed_size, hidden_size, num_layers, dropout, cell_type="GRU"):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(input_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        
        if cell_type == "GRU":
            self.rnn = nn.GRU(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        else:  # Default to RNN
            self.rnn = nn.RNN(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        
        self.cell_type = cell_type
    
    def forward(self, x, lengths):
        # x shape: (batch_size, seq_length)
        embedded = self.dropout(self.embedding(x))
        
        # Pack padded batch of sequences for RNN
        packed = nn.utils.rnn.pack_padded_sequence(
            embedded, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        
        # Forward propagate RNN
        outputs, hidden = self.rnn(packed)
        
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        
        return outputs, hidden


class Decoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers, dropout, cell_type="GRU"):
        super(Decoder, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        
        self.embedding = nn.Embedding(output_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        
        if cell_type == "GRU":
            self.rnn = nn.GRU(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        else:  # Default to RNN
            self.rnn = nn.RNN(
                embed_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        
        self.fc_out = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden):
        # x shape: (batch_size, 1)
        embedded = self.dropout(self.embedding(x))
        
        # Forward propagate through RNN
        output, hidden = self.rnn(embedded, hidden)
        
        # Compute output
        prediction = self.fc_out(output.squeeze(1))
        
        return prediction, hidden


class Seq2Seq(nn.Module):
    def __init__(self, 
                 input_size, 
                 output_size, 
                 embed_size, 
                 hidden_size, 
                 num_encoder_layers, 
                 num_decoder_layers, 
                 dropout, 
                 cell_type="GRU"):
        super(Seq2Seq, self).__init__()
        
        self.encoder = Encoder(
            input_size, 
            embed_size, 
            hidden_size, 
            num_encoder_layers, 
            dropout, 
            cell_type
        )
        
        self.decoder = Decoder(
            output_size, 
            embed_size, 
            hidden_size, 
            num_decoder_layers, 
            dropout, 
            cell_type
        )
        
        self.cell_type = cell_type
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.hidden_size = hidden_size
    
    def forward(self, source, target, source_lengths, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_vocab_size = self.decoder.output_size
        
        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(source.device)
        
        # Encode source sequences
        encoder_outputs, hidden = self.encoder(source, source_lengths)
        
        # Process hidden state if encoder and decoder layers differ
        hidden = self._process_hidden_for_decoder(hidden, batch_size)
        
        # First input to the decoder is the < SOS > token
        decoder_input = target[:, 0].unsqueeze(1)
        
        # Decode one character at a time
        for t in range(1, target_len):
            output, hidden = self.decoder(decoder_input, hidden)
            outputs[:, t, :] = output
            
            # Teacher forcing: decide whether to use the predicted or actual target as next input
            teacher_force = random.random() < teacher_forcing_ratio
            
            # Get the highest predicted token
            top1 = output.argmax(1)
            
            # If teacher forcing, use actual next token as input; otherwise use predicted token
            decoder_input = target[:, t].unsqueeze(1) if teacher_force else top1.unsqueeze(1)
        
        return outputs
    
    def _process_hidden_for_decoder(self, hidden, batch_size):
        """Process the encoder hidden state for the decoder"""
        if self.num_encoder_layers == self.num_decoder_layers:
            return hidden
        
        # For GRU and RNN
        if self.cell_type != "LSTM":
            if self.num_encoder_layers < self.num_decoder_layers:
                # Duplicate last layer to match decoder layers
                additional_layers = self.num_decoder_layers - self.num_encoder_layers
                last_layer = hidden[-1:].expand(additional_layers, batch_size, self.hidden_size)
                return torch.cat([hidden, last_layer], dim=0)
            else:
                # Use only the last n layers
                return hidden[-self.num_decoder_layers:]
        else:
            # For LSTM (hidden is a tuple of (h_0, c_0))
            h, c = hidden
            if self.num_encoder_layers < self.num_decoder_layers:
                # Duplicate last layer to match decoder layers
                additional_layers = self.num_decoder_layers - self.num_encoder_layers
                last_h_layer = h[-1:].expand(additional_layers, batch_size, self.hidden_size)
                last_c_layer = c[-1:].expand(additional_layers, batch_size, self.hidden_size)
                new_h = torch.cat([h, last_h_layer], dim=0)
                new_c = torch.cat([c, last_c_layer], dim=0)
                return (new_h, new_c)
            else:
                # Use only the last n layers
                return (h[-self.num_decoder_layers:], c[-self.num_decoder_layers:])
    
    def predict(self, source, source_len, target_vocab_size, sos_idx, eos_idx, max_len=100):
        batch_size = source.shape[0]
        
        # Encode source sequences
        encoder_outputs, hidden = self.encoder(source, source_len)
        
        # Process hidden state if encoder and decoder layers differ
        hidden = self._process_hidden_for_decoder(hidden, batch_size)
        
        # First input to the decoder is the < SOS > token
        decoder_input = torch.tensor([[sos_idx]] * batch_size).to(source.device)
        
        # Lists to store predicted outputs
        predictions = []
        
        # Flag to indicate if decoding is complete
        done = [False] * batch_size
        
        # Decode one character at a time
        for _ in range(max_len):
            output, hidden = self.decoder(decoder_input, hidden)
            
            # Get the highest predicted token
            top1 = output.argmax(1)
            
            # Store predicted token
            predictions.append(top1.unsqueeze(1))
            
            # Check if all sequences have reached EOS
            for i in range(batch_size):
                if top1[i].item() == eos_idx:
                    done[i] = True
            
            if all(done):
                break
            
            # Use predicted token as next input
            decoder_input = top1.unsqueeze(1)
        
        # Concatenate predictions along the sequence dimension
        return torch.cat(predictions, dim=1)

##### Attention based Seq2seq model using pytorch

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

class Attention(nn.Module):
    def __init__(self, encoder_hidden_size, decoder_hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(encoder_hidden_size + decoder_hidden_size, decoder_hidden_size)
        self.v = nn.Linear(decoder_hidden_size, 1, bias=False)
        
    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        # Reshape hidden to match encoder_outputs for concatenation
        # For GRU/RNN: hidden is [num_layers, batch_size, hidden_size]
        # For LSTM: hidden is a tuple of [h, c], each [num_layers, batch_size, hidden_size]
        
        # Handle both LSTM and GRU/RNN cases
        if isinstance(hidden, tuple):  # LSTM case
            hidden_state = hidden[0]  # Use the hidden state (h), not cell state (c)
        else:  # GRU/RNN case
            hidden_state = hidden
            
        # Take the last layer's hidden state and expand it to match encoder_outputs length
        last_hidden = hidden_state[-1].unsqueeze(1).expand(-1, src_len, -1)
        
        # Now hidden and encoder_outputs should have compatible dimensions for concatenation
        # last_hidden: [batch_size, src_len, hidden_size]
        # encoder_outputs: [batch_size, src_len, hidden_size]
        energy = torch.tanh(self.attn(torch.cat((last_hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        
        return F.softmax(attention, dim=1)

class AttentionDecoder(nn.Module):
    def __init__(self, output_size, embed_size, hidden_size, num_layers, dropout, cell_type="GRU"):
        super(AttentionDecoder, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        
        self.embedding = nn.Embedding(output_size, embed_size)
        self.dropout = nn.Dropout(dropout)
        self.attention = Attention(hidden_size, hidden_size)
        
        if cell_type == "GRU":
            self.rnn = nn.GRU(
                embed_size + hidden_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        elif cell_type == "LSTM":
            self.rnn = nn.LSTM(
                embed_size + hidden_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        else:
            self.rnn = nn.RNN(
                embed_size + hidden_size,
                hidden_size,
                num_layers,
                dropout=dropout if num_layers > 1 else 0,
                batch_first=True
            )
        
        self.fc_out = nn.Linear(hidden_size * 2 + embed_size, output_size)
    
    def forward(self, x, hidden, encoder_outputs):
        x = x.unsqueeze(1)
        embedded = self.dropout(self.embedding(x))
        
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        
        weighted = torch.bmm(a, encoder_outputs)
        
        rnn_input = torch.cat((embedded, weighted), dim=2)
        
        output, hidden = self.rnn(rnn_input, hidden)
        
        output = output.squeeze(1)
        weighted = weighted.squeeze(1)
        embedded = embedded.squeeze(1)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        
        return prediction, hidden, a.squeeze(1)

class Seq2SeqAttention(nn.Module):
    def __init__(self, 
                 input_size, 
                 output_size, 
                 embed_size, 
                 hidden_size, 
                 num_encoder_layers, 
                 num_decoder_layers, 
                 dropout, 
                 cell_type="GRU"):
        super(Seq2SeqAttention, self).__init__()
        
        #from seq2seq import Encoder
        
        self.encoder = Encoder(
            input_size, 
            embed_size, 
            hidden_size, 
            num_encoder_layers, 
            dropout, 
            cell_type
        )
        
        self.decoder = AttentionDecoder(
            output_size, 
            embed_size, 
            hidden_size, 
            num_decoder_layers, 
            dropout, 
            cell_type
        )
        
        self.cell_type = cell_type
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.hidden_size = hidden_size
    
    def forward(self, source, target, source_lengths, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_vocab_size = self.decoder.output_size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(source.device)
        attentions = torch.zeros(batch_size, target_len, source.shape[1]).to(source.device)
        
        encoder_outputs, hidden = self.encoder(source, source_lengths)
        
        hidden = self._process_hidden_for_decoder(hidden, batch_size)
        
        decoder_input = target[:, 0]
        
        for t in range(1, target_len):
            output, hidden, attention = self.decoder(decoder_input, hidden, encoder_outputs)
            outputs[:, t, :] = output
            attentions[:, t, :] = attention
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(1)
            
            decoder_input = target[:, t] if teacher_force else top1
        
        return outputs, attentions
    
    def _process_hidden_for_decoder(self, hidden, batch_size):
        if self.num_encoder_layers == self.num_decoder_layers:
            return hidden
        
        if self.cell_type != "LSTM":
            if self.num_encoder_layers < self.num_decoder_layers:
                additional_layers = self.num_decoder_layers - self.num_encoder_layers
                last_layer = hidden[-1:].expand(additional_layers, batch_size, self.hidden_size)
                return torch.cat([hidden, last_layer], dim=0)
            else:
                return hidden[-self.num_decoder_layers:]
        else:
            h, c = hidden
            if self.num_encoder_layers < self.num_decoder_layers:
                additional_layers = self.num_decoder_layers - self.num_encoder_layers
                last_h_layer = h[-1:].expand(additional_layers, batch_size, self.hidden_size)
                last_c_layer = c[-1:].expand(additional_layers, batch_size, self.hidden_size)
                new_h = torch.cat([h, last_h_layer], dim=0)
                new_c = torch.cat([c, last_c_layer], dim=0)
                return (new_h, new_c)
            else:
                return (h[-self.num_decoder_layers:], c[-self.num_decoder_layers:])
    
    def predict(self, source, source_len, target_vocab_size, sos_idx, eos_idx, max_len=100):
        batch_size = source.shape[0]
        
        encoder_outputs, hidden = self.encoder(source, source_len)
        
        hidden = self._process_hidden_for_decoder(hidden, batch_size)
        
        decoder_input = torch.tensor([sos_idx] * batch_size).to(source.device)
        
        predictions = []
        attention_weights = []
        
        done = [False] * batch_size
        
        for _ in range(max_len):
            output, hidden, attention = self.decoder(decoder_input, hidden, encoder_outputs)
            
            top1 = output.argmax(1)
            
            predictions.append(top1.unsqueeze(1))
            attention_weights.append(attention.unsqueeze(1))
            
            for i in range(batch_size):
                if top1[i].item() == eos_idx:
                    done[i] = True
            
            if all(done):
                break
            
            decoder_input = top1
        
        return torch.cat(predictions, dim=1), torch.cat(attention_weights, dim=1)

##### utilities function for calculating accuracy and checkpoints

In [13]:
import torch
import os
import numpy as np

def calculate_accuracy(predictions, targets, target_lengths, ignore_index=0):
    """
    Calculate the accuracy of the predictions against the targets.
    A prediction is considered correct only if all characters in the sequence match.
    """
    batch_size = predictions.size(0)
    correct = 0
    
    for i in range(batch_size):
        pred_seq = predictions[i].cpu().numpy()
        target_seq = targets[i, 1:target_lengths[i]-1].cpu().numpy()  # Exclude SOS and EOS tokens
        
        # Check if prediction exactly matches target
        if len(pred_seq) == len(target_seq) and np.array_equal(pred_seq, target_seq):
            correct += 1
    
    return correct / batch_size

def save_checkpoint(model, optimizer, epoch, accuracy, filename):
    """Save model checkpoint"""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'accuracy': accuracy
    }, filename)

def load_checkpoint(model, optimizer, filename):
    """Load model checkpoint"""
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    accuracy = checkpoint.get('accuracy', 0.0)
    return model, optimizer, epoch, accuracy

def indices_to_string(indices, idx_to_char, eos_idx=None):
    """Convert a sequence of indices to a string"""
    if eos_idx is not None:
        # Find the index of the first EOS token
        try:
            eos_pos = indices.index(eos_idx)
            indices = indices[:eos_pos]
        except ValueError:
            pass  # No EOS token found
    
    return ''.join([idx_to_char[idx] for idx in indices if idx in idx_to_char and idx_to_char[idx] not in ['<PAD>', '< SOS >', '<EOS>']])

def create_directory(directory):
    """Create directory if it doesn't exist"""
    if not os.path.exists(directory):
        os.makedirs(directory)

Config contains (Data parameter,Model Parameter and Training Parameter)

In [14]:
import os

class Config:
    # Data parameters
    language = os.environ.get("TRANSLITERATION_LANGUAGE", "hi")  # Get language from env var or default to "hi"
    #base_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of the current file
    #data_dir = os.path.join(base_dir, "dakshina_dataset_v1.0", f"{language}", "lexicons") 
    input_dir = "/kaggle/input/dataset"  # Read-only input path
    base_dir = "/kaggle/working"         # Writable path for outputs
    
    data_dir = os.path.join(input_dir, "dakshina_dataset_v1.0", language, "lexicons")
    # Check if data directory exists, if not, try to find it elsewhere
    if not os.path.exists(data_dir):
        # Try to find the dakshina dataset in parent directories
        parent_dir = os.path.dirname(base_dir)
        alt_data_dir = os.path.join(parent_dir, "dakshina_dataset_v1.0", f"{language}", "lexicons")
        if os.path.exists(alt_data_dir):
            data_dir = alt_data_dir
    
    train_file = os.path.join(data_dir, f"{language}.translit.sampled.train.tsv")
    val_file = os.path.join(data_dir, f"{language}.translit.sampled.dev.tsv")
    test_file = os.path.join(data_dir, f"{language}.translit.sampled.test.tsv")
    
    # Model parameters (default values)
    embed_size = 64
    hidden_size = 128
    num_encoder_layers = 1
    num_decoder_layers = 1
    dropout = 0.2
    cell_type = "GRU"  # Options: RNN, LSTM, GRU
    
    # Training parameters
    batch_size = 64
    epochs = 20
    learning_rate = 0.001
    teacher_forcing_ratio = 0.5
    
    # Decoding parameters
    beam_size = 1  # 1 for greedy decoding
    
    # W&B parameters
    wandb_project = "DL_A3_seq2seq"
    wandb_entity = "da24m006-iit-madras"  
    
    # Save directories
    model_dir = os.path.join(base_dir, "saved_models", language)
    # Create directory for saving model if it doesn't exist
    if not os.path.exists(model_dir):
        os.makedirs(model_dir, exist_ok=True)
    
    prediction_dir = os.path.join(base_dir, "predictions", language)
    # Create directory for saving prediction if it doesn't exist
    if not os.path.exists(prediction_dir):
        os.makedirs(prediction_dir, exist_ok=True)

    
    def __init__(self):
        # Verify paths and print info
        print(f"Base directory: {self.base_dir}")
        print(f"Language: {self.language}")
        print(f"Data directory: {self.data_dir}")
        print(f"Train file exists: {os.path.exists(self.train_file)}")
        print(f"Val file exists: {os.path.exists(self.val_file)}")
        print(f"Test file exists: {os.path.exists(self.test_file)}")
        
        # Print sample data from files if they exist
        self._print_sample_data(self.train_file, "train")
        self._print_sample_data(self.val_file, "validation")
        self._print_sample_data(self.test_file, "test")
    
    def _print_sample_data(self, file_path, name):
        """Print a sample of data from the file for debugging"""
        if os.path.exists(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    lines = f.readlines()[:3]  # Get first 3 lines
                    if lines:
                        print(f"\nSample {name} data:")
                        for line in lines:
                            print(f"  {line.strip()}")
                        print(f"  ... (total lines: {sum(1 for _ in open(file_path, 'r', encoding='utf-8'))})")
                    else:
                        print(f"\nWarning: {name} file exists but is empty: {file_path}")
            except Exception as e:
                print(f"\nError reading {name} file: {str(e)}")

In [15]:
def train_attention(config, run_name=None, init_wandb=True):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Only initialize wandb if specified
    if init_wandb:
        if run_name:
            wandb.init(project=config.wandb_project, entity=config.wandb_entity, name=run_name, config=vars(config))
        else:
            default_run_name = f"attn_{config.cell_type}_{config.language}_e{config.embed_size}_h{config.hidden_size}"
            wandb.init(project=config.wandb_project, entity=config.wandb_entity, name=default_run_name, config=vars(config))
        
        wandb.config.update({"language": config.language})
    
    print(f"\nTraining attention-based transliteration model for language: {config.language}")
    print(f"Using dataset from: {config.data_dir}\n")
    
    train_loader, train_dataset = get_dataloader(
        config.train_file, 
        config.batch_size, 
        shuffle=True
    )
    
    val_loader, val_dataset = get_dataloader(
        config.val_file, 
        config.batch_size, 
        shuffle=False
    )
    
    model = Seq2SeqAttention(
        input_size=train_dataset.source_vocab_size,
        output_size=train_dataset.target_vocab_size,
        embed_size=config.embed_size,
        hidden_size=config.hidden_size,
        num_encoder_layers=config.num_encoder_layers,
        num_decoder_layers=config.num_decoder_layers,
        dropout=config.dropout,
        cell_type=config.cell_type
    )
    
    model = model.to(device)
    
    criterion = torch.nn.CrossEntropyLoss(ignore_index=0)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
    
    create_directory(config.model_dir)
    
    best_accuracy = 0.0
    
    for epoch in range(config.epochs):
        model.train()
        total_loss = 0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{config.epochs}')
        
        for batch_idx, batch in enumerate(pbar):
            source = batch['source'].to(device)
            target = batch['target'].to(device)
            source_lengths = batch['source_lengths']
            
            optimizer.zero_grad()
            
            output, _ = model(source, target, source_lengths, config.teacher_forcing_ratio)
            
            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            target = target[:, 1:].reshape(-1)
            
            loss = criterion(output, target)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': loss.item()})
        
        avg_loss = total_loss / len(train_loader)
        
        model.eval()
        val_loss, val_accuracy = evaluate_attention(model, val_loader, criterion, device, train_dataset)
        
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': avg_loss,
            'val_loss': val_loss,
            'val_accuracy': val_accuracy,
            'language': config.language
        })
        
        print(f'Epoch: {epoch+1}, Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
        
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            if not os.path.exists(os.path.join(config.model_dir)):
                os.makedirs(os.path.join(config.model_dir))

            model_path = os.path.join(config.model_dir, f'best_attention_model_{config.language}_{wandb.run.id}.pt')
            save_checkpoint(
                model, 
                optimizer, 
                epoch, 
                val_accuracy, 
                model_path
            )
            
            try:
                wandb_model_path = os.path.join(wandb.run.dir, "best_attention_model.pt")
                torch.save(model.state_dict(), wandb_model_path)
                
                artifact = wandb.Artifact(f"attention-model-{config.language}-{wandb.run.id}", type="model")
                artifact.add_file(wandb_model_path)
                wandb.log_artifact(artifact)
                
                print(f"Model successfully saved to W&B as artifact")
            except Exception as e:
                print(f"Failed to save model to W&B: {str(e)}")
                print(f"Model was saved locally to {model_path}")
        
        if (epoch + 1) % 5 == 0:
            checkpoint_path = os.path.join(config.model_dir, f'checkpoint_attention_{config.language}_{epoch+1}_{wandb.run.id}.pt')
            save_checkpoint(
                model, 
                optimizer, 
                epoch, 
                val_accuracy, 
                checkpoint_path
            )
    
    # Only finish wandb if we initialized it
    if init_wandb:
        wandb.finish()
        
    return model, best_accuracy

def evaluate_attention(model, dataloader, criterion, device, dataset):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_examples = 0
    
    with torch.no_grad():
        for batch in dataloader:
            source = batch['source'].to(device)
            target = batch['target'].to(device)
            source_lengths = batch['source_lengths']
            
            output, _ = model(source, target, source_lengths, teacher_forcing_ratio=0)
            
            output_dim = output.shape[-1]
            output_for_loss = output[:, 1:].reshape(-1, output_dim)
            target_for_loss = target[:, 1:].reshape(-1)
            
            loss = criterion(output_for_loss, target_for_loss)
            total_loss += loss.item()
            
            predictions, _ = model.predict(
                source,
                source_lengths,
                dataset.target_vocab_size,
                dataset.target_char_to_idx['< SOS >'],
                dataset.target_char_to_idx['<EOS>']
            )
            
            for i in range(len(predictions)):
                pred_seq = [idx.item() for idx in predictions[i]]
                pred_str = indices_to_string(pred_seq, dataset.target_idx_to_char, dataset.target_char_to_idx['<EOS>'])
                target_str = batch['target_texts'][i]
                
                if pred_str == target_str:
                    total_correct += 1
            
            total_examples += len(predictions)
    
    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_examples
    
    return avg_loss, accuracy

# if __name__ == "__main__":
#     config = Config()
#     train_attention(config)

In [17]:
sweep_attention_configuration = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'embed_size': {
            'values': [32, 64, 128, 256]
        },
        'hidden_size': {
            'values': [64, 128, 256, 512]
        },
        'num_encoder_layers': {
            'values': [1, 2]
        },
        'num_decoder_layers': {
            'values': [1, 2]
        },
        'cell_type': {
            'values': ['GRU', 'LSTM']
        },
        'dropout': {
            'values': [0.2, 0.3, 0.5]
        },
        'learning_rate': {
            'values': [0.0001, 0.001, 0.002]
        },
        'batch_size': {
            'values': [64, 128]
        },
        'teacher_forcing_ratio': {
            'values': [0.5, 0.7, 0.9]
        }
    }
}

def sweep_train_attention(config_defaults=None):
    config = Config()  # initialize config early to construct run_name

    # Construct initial run name from default values
    run_name = f"attn_{config.language}_{config.cell_type}_drop{config.dropout}_es{config.embed_size}_hs{config.hidden_size}_en{config.num_encoder_layers}_de{config.num_decoder_layers}_lr{config.learning_rate}"
    
    with wandb.init(config=config_defaults, name=run_name) as run:
        wandb_config = wandb.config
        for key, value in wandb_config.items():
            if hasattr(config, key):
                setattr(config, key, value)

        # Rebuild run name with updated sweep values
        run_name = f"attn_{config.language}_{config.cell_type}_drop{config.dropout}_es{config.embed_size}_hs{config.hidden_size}_en{config.num_encoder_layers}_de{config.num_decoder_layers}_lr{config.learning_rate}"
        wandb.run.name = run_name
        wandb.run.save()

        # Train the model — avoid reinitializing wandb
        model, accuracy = train_attention(config, run_name=run_name, init_wandb=False)

        # Log final accuracy
        run.summary['best_accuracy'] = accuracy

# def sweep_train_attention(config_defaults=None):
#     # The issue is here - we need to ensure wandb.init() is called before using wandb.summary
#     run = wandb.init(config=config_defaults)
    
#     config = Config()
    
#     # Update config with parameters from sweep
#     wandb_config = wandb.config
#     for key, value in wandb_config.items():
#         if hasattr(config, key):
#             setattr(config, key, value)
    
#     # Create model name based on parameters
#     run_name = f"attn_{config.language}_{config.cell_type}_e{config.embed_size}_h{config.hidden_size}_en{config.num_encoder_layers}_de{config.num_decoder_layers}_d{config.dropout}_lr{config.learning_rate}"
            
#     # Train the model
#     # Don't initialize wandb again in train_attention
#     model, accuracy = train_attention(config, run_name=run_name, init_wandb=False)
    
#     # Log the final result
#     wandb.run.summary['best_accuracy'] = accuracy
#     # No need to call wandb.finish() as the context manager will handle it


# def main():
#     parser = argparse.ArgumentParser(description='Run W&B sweep for attention transliteration model')
#     parser.add_argument('--sweep_id', type=str, help='The sweep ID to use (if already created)')
#     parser.add_argument('--count', type=int, default=20, help='Number of sweep runs')
#     parser.add_argument('--language', type=str, default='hi', 
#                         choices=['bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'pa', 'sd', 'si', 'ta', 'te', 'ur'],
#                         help='Language code to use (default: hi)')
#     args = parser.parse_args()
    
#     # Set language in environment variable
#     os.environ["TRANSLITERATION_LANGUAGE"] = args.language
    
#     # Set up W&B project
#     config = Config()
    
#     if args.sweep_id:
#         sweep_id = args.sweep_id
#     else:
#         sweep_id = wandb.sweep(sweep=sweep_attention_configuration, project=config.wandb_project, entity=config.wandb_entity)
    
#     # Start sweep agent
#     wandb.agent(sweep_id, function=sweep_train_attention, count=args.count)

# if __name__ == "__main__":
#     main()

os.environ["TRANSLITERATION_LANGUAGE"] = "hi"
config = Config()

sweep_id = wandb.sweep(sweep=sweep_attention_configuration, project=config.wandb_project, entity=config.wandb_entity)
wandb.agent(sweep_id, function=sweep_train_attention, count=30)

Base directory: /kaggle/working
Language: hi
Data directory: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons
Train file exists: True
Val file exists: True
Test file exists: True

Sample train data:
  अं	an	3
  अंकगणित	ankganit	3
  अंकल	uncle	4
  ... (total lines: 44204)

Sample validation data:
  अंकन	ankan	3
  अंगकोर	angkor	3
  अंगिरा	angira	3
  ... (total lines: 4358)

Sample test data:
  अंक	ank	5
  अंक	anka	1
  अंकित	ankit	3
  ... (total lines: 4502)
Create sweep with ID: ij9kejdj
Sweep URL: https://wandb.ai/da24m006-iit-madras/DL_A3_seq2seq/sweeps/ij9kejdj


wandb: Agent Starting Run: yx3v0jfb with config:
wandb: 	batch_size: 64
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	embed_size: 64
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.002
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 1
wandb: 	teacher_forcing_ratio: 0.7


Base directory: /kaggle/working
Language: hi
Data directory: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons
Train file exists: True
Val file exists: True
Test file exists: True

Sample train data:
  अं	an	3
  अंकगणित	ankganit	3
  अंकल	uncle	4
  ... (total lines: 44204)

Sample validation data:
  अंकन	ankan	3
  अंगकोर	angkor	3
  अंगिरा	angira	3
  ... (total lines: 4358)

Sample test data:
  अंक	ank	5
  अंक	anka	1
  अंकित	ankit	3
  ... (total lines: 4502)


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.



Training attention-based transliteration model for language: hi
Using dataset from: /kaggle/input/dataset/dakshina_dataset_v1.0/hi/lexicons



...

Epoch 19/20: 100%|██████████| 691/691 [00:25<00:00, 26.79it/s, loss=0.306] 


Epoch: 19, Train Loss: 0.1742, Val Loss: 13.7959, Val Accuracy: 0.3922


Epoch 20/20: 100%|██████████| 691/691 [00:25<00:00, 26.77it/s, loss=0.316] 


Epoch: 20, Train Loss: 0.1721, Val Loss: 13.9337, Val Accuracy: 0.3917


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▇▇▇▇██▇▇▇▇▇▇▇▇▇▆▆
val_loss,▁▂▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇██
best_accuracy,0.42244
epoch,20
language,hi
train_loss,0.17209
val_accuracy,0.39169
val_loss,13.93369
